In [1]:
#!pip install pyspark
#!pip install --upgrade google-cloud-bigquery[pandas]
#!pip install pyspark[sql] #PARSEO RAPIDO DE PANDAS A SPARK RDDSQL

### Importing the libraries and starting the Spark Session

In [31]:
import pyspark.sql.functions as sql_func
from pyspark.sql.types import *
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.context import SparkContext 
from pyspark.sql import SparkSession
from pyspark.mllib.evaluation import RegressionMetrics, RankingMetrics
from pyspark.ml.evaluation import RegressionEvaluator
import pandas as pd

In [3]:
#!export ARROW_PRE_0_15_IPC_FORMAT=1
#!echo $ARROW_PRE_0_15_IPC_FORMAT

In [4]:
sc = SparkContext.getOrCreate()

In [5]:
spark = SparkSession(sc)
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [6]:
#from pyspark.context import 
spark = SparkSession(sc)

### Add asset from remote connection 

In [7]:
final_stat = None

In [8]:
# Get data from GCP- BQ
from google.cloud import bigquery
import time
t0 = time.time()

porcentaje = 9

limite = int(189857 * porcentaje)

def get_data_BQ(sql):
    client = bigquery.Client()
    df = client.query(sql).to_dataframe()
    return(df)
sql =  '''SELECT USERID as ID_CTE, ID_FAM as ID_CLAS1, FREQUENCY as FREQUENCY
FROM `rmf2gcp.RawData.Workflow_aggregado`
WHERE id_table_dem <= ''' + str(limite) #310 588 606 ''' # corre en mi local y pesa 56MB %1 del total de la muestra
print(sql)

SELECT USERID as ID_CTE, ID_FAM as ID_CLAS1, FREQUENCY as FREQUENCY
FROM `rmf2gcp.RawData.Workflow_aggregado`
WHERE id_table_dem <= 1708713


In [ ]:
final_stat = get_data_BQ(sql)

In [ ]:
print(final_stat.dtypes)
print(final_stat.shape)

ID_CTE       int64
ID_CLAS1     int64
FREQUENCY    int64
dtype: object
(32639220, 3)


In [ ]:
final_stat = spark.createDataFrame(final_stat)
final_stat.show(5)

/usr/lib/spark/python/pyspark/sql/session.py:714: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  An error occurred while calling z:org.apache.spark.sql.api.python.PythonSQLUtils.readArrowStreamFromFile.
: java.lang.IllegalArgumentException
	at java.nio.ByteBuffer.allocate(ByteBuffer.java:334)
	at org.apache.arrow.vector.ipc.message.MessageSerializer.readMessage(MessageSerializer.java:543)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anon$3.readNextBatch(ArrowConverters.scala:243)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$$anon$3.<init>(ArrowConverters.scala:229)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$.getBatchesFromStream(ArrowConverters.scala:228)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$.$anonfun$readArrowStreamFromFile$2(ArrowConverters.scala:216)
	at org.apache.spark.util.Utils$.tryWithResource(Utils.scala:2543)


+-------+--------+---------+
| ID_CTE|ID_CLAS1|FREQUENCY|
+-------+--------+---------+
|7502746|  103079|        8|
|4393818|  313152|        6|
|7958247|  380283|        7|
|1574064|  380283|        7|
|7913849|  106061|        6|
+-------+--------+---------+
only showing top 5 rows



In [ ]:
final_stat.count()
print(type(final_stat))

<class 'pyspark.sql.dataframe.DataFrame'>


### Preparing data for the model

In [ ]:
ratings = (final_stat
    .select(
        'ID_CTE',
        'ID_CLAS1',
        'FREQUENCY',
    )
).cache()

### Spliting the data set to test and train for measuring the performance of the ALS Model

In [ ]:
(training, test) = ratings.randomSplit([0.8, 0.2])

### Build the recommendation model using ALS on the training data


In [ ]:
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=2, regParam=0.01, 
          userCol="ID_CTE", itemCol="ID_CLAS1", ratingCol="FREQUENCY",
          coldStartStrategy="drop",
          implicitPrefs=True)

model = als.fit(ratings)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="FREQUENCY",
                                predictionCol="prediction")

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.5462754409847685


In [ ]:
predictions.show(5)

+-------+--------+---------+------------+
| ID_CTE|ID_CLAS1|FREQUENCY|  prediction|
+-------+--------+---------+------------+
|4056813|  212010|        1|4.5328977E-4|
|8226344|  212010|        1|6.6993537E-4|
|6301469|  212010|        1|1.8051936E-4|
|8203832|  212010|        1|  5.51962E-4|
|5656025|  212010|        1| 3.318036E-4|
+-------+--------+---------+------------+
only showing top 5 rows



## Parameters of ALS Model in PySpark realization are following:

##### NumBlocks is the number of blocks the users and items will be partitioned into in order to parallelize computation.
##### rank is the number of latent factors in the model.
##### maxIter is the maximum number of iterations to run.
##### regParam specifies the regularization parameter in ALS.
##### implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data (defaults to false which means using explicit feedback).
##### alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations (defaults to 1.0)

###  Generate top 10 Item recommendations for each user



In [ ]:
userRecs = model.recommendForAllUsers(10)
print(userRecs.count())
userRecs.show(5)

1708713
+------+--------------------+
|ID_CTE|     recommendations|
+------+--------------------+
| 10362|[[224025, 0.05139...|
| 11033|[[701305, 0.53538...|
| 11141|[[105068, 0.37909...|
| 12940|[[380283, 0.77650...|
| 13832|[[318009, 0.55853...|
+------+--------------------+
only showing top 5 rows



In [ ]:
userRecs.take(2)

[Row(ID_CTE=10362, recommendations=[Row(ID_CLAS1=224025, rating=0.05139085650444031), Row(ID_CLAS1=229022, rating=0.04640220105648041), Row(ID_CLAS1=224041, rating=0.04161088541150093), Row(ID_CLAS1=102089, rating=0.03968541696667671), Row(ID_CLAS1=102016, rating=0.03477980196475983), Row(ID_CLAS1=290059, rating=0.03472079336643219), Row(ID_CLAS1=102164, rating=0.031431473791599274), Row(ID_CLAS1=701305, rating=0.030639003962278366), Row(ID_CLAS1=229035, rating=0.030155781656503677), Row(ID_CLAS1=102010, rating=0.02981329895555973)]),
 Row(ID_CTE=11033, recommendations=[Row(ID_CLAS1=701305, rating=0.5353883504867554), Row(ID_CLAS1=313064, rating=0.38923436403274536), Row(ID_CLAS1=314064, rating=0.35390523076057434), Row(ID_CLAS1=862009, rating=0.34793874621391296), Row(ID_CLAS1=381009, rating=0.3431849181652069), Row(ID_CLAS1=867047, rating=0.3318749964237213), Row(ID_CLAS1=314129, rating=0.2766352593898773), Row(ID_CLAS1=102089, rating=0.25155359506607056), Row(ID_CLAS1=423132, rating

In [ ]:
userRecs[['recommendations']].show()

+--------------------+
|     recommendations|
+--------------------+
|[[224025, 0.05139...|
|[[701305, 0.53538...|
|[[105068, 0.37909...|
|[[380283, 0.77650...|
|[[318009, 0.55853...|
|[[862009, 0.38653...|
|[[381009, 0.55220...|
|[[867048, 0.28203...|
|[[224073, 0.67350...|
|[[701305, 1.40924...|
|[[314156, 0.65680...|
|[[313152, 0.64131...|
|[[295019, 0.15151...|
|[[102089, 0.12812...|
|[[701305, 0.37526...|
|[[229032, 0.61908...|
|[[314156, 0.63714...|
|[[106055, 0.33072...|
|[[701305, 1.11097...|
|[[380073, 0.59425...|
+--------------------+
only showing top 20 rows



In [ ]:
1

1

### Display the recommendations and get them in the correct format

In [ ]:
from pyspark.sql.functions import explode
userRecs1=userRecs.withColumn("recommendations", explode(userRecs.recommendations))
userRecs1.show(4)

+------+--------------------+
|ID_CTE|     recommendations|
+------+--------------------+
| 10362|[224025, 0.051390...|
| 10362| [229022, 0.0464022]|
| 10362|[224041, 0.041610...|
| 10362|[102089, 0.039685...|
+------+--------------------+
only showing top 4 rows



####  Breaking down reach recommendation to separate columns

In [ ]:
userRecs1= userRecs1.select('ID_CTE', 'recommendations.*')       

### Display the results

In [ ]:
userRecs1.show(2) 

+------+--------+-----------+
|ID_CTE|ID_CLAS1|     rating|
+------+--------+-----------+
| 10362|  224025|0.051390857|
| 10362|  229022|  0.0464022|
+------+--------+-----------+
only showing top 2 rows



In [ ]:
userRecs1.count()

17087130

### Writing the Output back to the Remote Datasource

In [ ]:
final_stat = userRecs1.toPandas()
userRecs1.unpersist(True)

DataFrame[ID_CTE: int, ID_CLAS1: int, rating: float]

In [30]:
!pip install pandas_gbq

In [ ]:
table_id = 'Resultados.test_spark_0'+str(porcentaje)+'porciento_17_junio_2020'
table_id

'Resultados.test_spark_09porciento_17_junio_2020'

In [ ]:
final_stat.to_gbq(table_id, project_id='rmf2gcp')
t3 = time.time()
total = t3-t0
print(total)

1it [03:39, 219.13s/it]

3510.216548681259


In [ ]:
#!mkdir test/

In [ ]:
#final_stat.to_csv('test_spark_0'+str(porcentaje)+'porciento_17_junio_2020')

In [ ]:
#!gsutil cp test_gcp_cluster_10_junio_2020.csv gs://resultadosrmf2/prueba_gcp_01porciento/test_local_10_junio_2020.csv

In [ ]:
#!zip test_gcp_cluster_10_junio_2020.csv.zip test_gcp_cluster_10_junio_2020.csv

In [ ]:
#!ls

In [ ]:
#!gsutil cp test_gcp_cluster_10_junio_2020.csv.zip gs://resultadosrmf2/prueba_gcp_01porciento/test_local_10_junio_2020.csv.zip

In [ ]:
##!rm -r test_modelos
#!mkdir test_modelos_gcp
#!chmod 777 test_modelos_gcp

In [ ]:
#from pyspark.ml import Pipeline

In [ ]:
#pipeline = Pipeline(stages=[model])

In [ ]:
#model_alsWML = pipeline.fit(ratings)

In [ ]:
#model_alsWML.save('/test_modelos_gcp/')

In [ ]:
#!ls -la

In [ ]:
#!gsutil cp test_gcp_cluster_10_junio_2020.csv.zip gs://resultadosrmf2/prueba_gcp_01porciento/test_local_10_junio_2020.csv.zip